# Importing packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

# Fill in the missing code
The following script is an implementation of a logistic regression model for a binary classification problem. The dataset contains four numerical input attributes and one output attribute (class label 0 or 1).

- Columns **Attribute1**, **Attribute2**, **Attribute3**, and **Attribute4** are inputs.
- Column **OutputClass** is output.

The missing pieces of code are indicated like so: `#write your code here#`.

The functions corresponding to said missing code are stated in the description above each function.

To complete the task, replace `#write your code here#` with the your own code. Also, ensure that the program executes without any warnings or errors.

In [ ]:
# Reading CSV file as pandas dataframe
data = pd.read_csv("/kaggle/input/summer-internship-cistup-iisc-2023/Full_Data.csv")

data_X = data.values[:, :-1]

data_Y = data.values[:, -1].astype("int")

# Splitting dataset into training and test set
X_full, test_X_full, Y, test_Y = train_test_split(data_X, data_Y, test_size=0.4, shuffle=True, random_state=1)

Try out different input column indexes as input data. For instance, if you choose to use input columns 2 and 3; replace `#write your code here#` with `1, 2` (resp.) in the code block below.

Example:
* `attr1 = 1`
* `attr2 = 2`

Note column 1 (i.e., column index = 0) is `RowIdx`, and should not be used as an input.

Also note, *column index = column number - 1*.

In [ ]:
# Removing all columns other than two
attr1 = 2
attr2 = 3
X = X_full[:, (attr1, attr2)]
test_X = test_X_full[:, (attr1, attr2)]

In [ ]:
# Ploting attributes
plt.plot(X[:, 0][Y==0], X[:, 1][Y==0], "o")
plt.plot(X[:, 0][Y==1], X[:, 1][Y==1], "s")
plt.xlabel("Attribute 1")
plt.ylabel("Attribute 2")

The code block below constructs $\mathbf{X}$. Note that $\mathbf{X}$ includes $x_0 = 1$ column for the bias (i.e., intercept).

In [ ]:
# Combining input data with fabricated output class
X = np.hstack((np.ones((X.shape[0], 1)), X))
print("The regenerated input data has class labels set to 1:\n", X)

## Logistic Regression Model

For Logistic Regression, our hypothesis is 
$$
\hat{Y} = h_w(x) = \frac{1}{1+e^{-(w^{T}x)}}
$$
The output range of $\hat{Y}$ is between 0 and 1.

### Sigmoid Function

The Sigmoid function squishes all its inputs (i.e., values on x-axis) between 0 and 1.
$$
\sigma(z) = \frac{1}{1+e^{-z}}
$$

In [ ]:
# Defining sigmoid function
def sigmoid(z):
    # z --> input
    # sigmoid_z --> output of sigmoid function
    z = z.astype(float)
    sigmoid_z = 1/(1+np.exp(-z))
    
    return sigmoid_z

#sample output
z=30
print_sig=sigmoid(np.array([z]))
print("sigmoid of",z,"is",print_sig)


The cost function for Logistic Regression for binary classification:
$$
J(data, w) = \frac{1}{n}\sum_{i=1}^{n} L(\hat{Y}^{(i)},Y^{(i)}) = -\frac{1}{n}\sum_{i=1}^{n} [Y^{(i)}log(\hat{Y}^{(i)}) + (1-Y^{(i)})log(1-\hat{Y}^{(i)})]
$$

This loss is also called binary cross entropy error.

In [ ]:
# Defining loss function
def loss(Y, y_hat):
    # Y --> data|
    # y_hat --> w
    loss_value = np.mean(Y * np.log(sigmoid(y_hat)) + (1 - Y) * np.log(1 - sigmoid(y_hat)))
    
    return -loss_value



### Gradient of the loss function

Using the Gradient Descent Algorithm, optimal values of the parameters can be calculated like so ($\eta$ →learning rate), the update rules for parameters are as follows:
$$
w_{t+1} = w_{t} - \eta*dw
$$
Where $dw$ is the partial derivative of loss w.r.t parameter $w$. It looks like:
$$
dw = \frac{1}{n} * (\hat{y}-y).\textbf{X}
$$

In [ ]:
# Defining gradient function
def gradients(X, Y, y_hat):
    # X --> input
    # Y --> true/target value
    # y_hat --> hypothesis/predictions
    # n --> number of training examples
    
    n = X.shape[0]
    
    # Gradient of loss w.r.t weights
    dw = (1/n) * np.dot((y_hat - Y).X)
    
    
    return dw

Normalize the data before using/computing gradient. It can accelerate the training process. Make sure you don"t normalize the "bias" term (i.e., first column).

In [ ]:
# Defining data normalization function
def normalize(X):
    # X --> input
    # n --> number of training examples
    # d --> number of features 
    n, d = X.shape
    
    # Normalizing all the d features of X (except the bias (first) column)
    for i in range(d-1):
        X[:,i+1] = (X[:,i+1] - X[:,i+1].mean(axis=0))/X[:,i+1].std(axis=0)
        
    return X

### Prediction

Now that the functions to learn the parameters are ready, check if the hypothesis ($\hat{Y}$) is able to predict the output class $Y=1$ or $Y=0$. Note that the hypothesis is the probability of $Y$ being 1 given $\textbf{X}$ and is parameterized by $w$.

Hence, the prediction function will be so —
$$
\hat{Y} = 1 \to w^{T}\textbf{X}
\geq 0
$$
$$
\hat{Y} = 0  \to w^{T}\textbf{X} < 0
$$

In [ ]:
# Defining prediction function
def predict(X,w):
    # X --> Input.
    
    # Normalizing the inputs.
    X = normalize(X)
    
    # Calculating prediction/y_hat.
    preds = sigmoid(np.dot(X, w))
 
    
    # Empty List to store predictions.
    
    pred_class=[]
    for pred in preds:
        if pred >= 1/2:
            pred_class.append(1)
        else:
            pred_class.append(0)
    
    return np.array(pred_class)
        
    

The decision boundary will be:
$$
\hat{Y} = 0.5 \quad or \quad w^{T}\textbf{X} = 0
$$

In [ ]:
# Defining function to plot decision boundary
def plot_decision_boundary(X,w):
    ydisp = -(w[0] + w[1] * X)/w[2]
    
    fig = plt.figure(figsize=(10, 8))
    plt.plot(X[:, 1][Y==0], X[:, 2][Y==0], "^")
    plt.plot(X[:, 1][Y==1], X[:, 2][Y==1], "s")
    
    plt.xlim([-2, 5])
    plt.ylim([-2, 5])
    plt.xlabel("Attribute 1")
    plt.ylabel("Attribute 2")
    plt.title("Decision Boundary")
    plt.plot(X, ydisp)

Now that all the required blocks for logistic regression model are ready, encode the model.

In [ ]:
# Defining training function
def train(X, Y, epochs, eta):
    # X --> input
    # Y --> true/target value
    # bs --> batch size
    # eta --> learning rate
    # n-> number of training examples
    # d-> number of features 
    
    n, d = X.shape
    
    # Initializing weights and bias to zeros
    w = np.zeros((d,1))
    
    # Reshaping Y
    Y = Y.reshape(n,1)
    
    # Normalizing the inputs
    X = normalize(X)
    
    # Empty list to store losses
    losses = []
    
    # Training loop
    for epoch in range(epochs):
        
            # Calculating hypothesis/prediction
            y_hat = sigmoid(np.dot(X, w))
            
            # Getting the gradients of loss w.r.t parameters
            dw =  (1/n) * np.dot(X.T, (y_hat - Y))
            # Updating the parameters.
            w = w- eta*dw
            
            # Calculating loss and appending it in the list
            l = loss(Y,y_hat)
            losses.append(l)
        
    # Returning weights, losses(List)
    return w, losses

In [ ]:
#Train the model and print the results.

#Try out different learning rates to improve model performance.

#Example: w, l = train(X, Y, epochs=100, eta=0.001)`

In [ ]:
# Training model 
w, l = train(X, Y, epochs=100, eta=0.001)



In [ ]:
# Plotting loss vs. epoch function
plt.plot(l)
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.show()


In [ ]:
# Printing training accuracy
print("The accuracy of model is",(np.sum(1*(Y==predict(X,w)))/len(Y))*100,"%")

In [ ]:
# Plotting the decision boundary
plot_decision_boundary(X, w)



In [ ]:
print(test_X.shape)

Run the test data through the trained model, and print the testing accuracy.

In [ ]:
# Checking test accuracy
test_X = np.hstack((np.ones((test_X.shape[0],1)), test_X))
ml_predictions = predict(test_X,w)
print("The test accuracy of model is",(np.sum(1*(test_Y==ml_predictions))/len(test_Y))*100,"%")


In [ ]:
# Exporting results
row_idx = np.concatenate((test_X_full[:, 0].astype(int), X_full[:, 0].astype(int)))
opt_cls = np.concatenate((ml_predictions.astype(int), Y.astype(int)))
                              
output_data = np.concatenate((row_idx.reshape(-1,1), opt_cls.reshape(-1,1)), axis=1)
pd.DataFrame(output_data, columns=["RowIdx", "OutputClass"]).to_csv("Output_Data.csv", index=False)